# Линейная регрессия: прогноз оклада по описанию вакансии

###### Введение
Линейные методы хорошо подходят для работы с разреженными данными — к таковым относятся, например, тексты. Это можно объяснить высокой скоростью обучения и небольшим количеством параметров, благодаря чему удается избежать переобучения.

Линейная регрессия имеет несколько разновидностей в зависимости от того, какой регуляризатор используется. Мы будем работать с гребневой регрессией, где применяется квадратичный, или L2-регуляризатор.

###### Реализация в Scikit-Learn
Для извлечения TF-IDF-признаков из текстов воспользуйтесь классом sklearn.feature_extraction.text.TfidfVectorizer. 

Для предсказания целевой переменной мы будем использовать гребневую регрессию, которая реализована в классе sklearn.linear_model.Ridge.

Обратите внимание, что признаки LocationNormalized и ContractTime являются строковыми, и поэтому с ними нельзя работать напрямую. Такие нечисловые признаки с неупорядоченными значениями называют категориальными или номинальными. Типичный подход к их обработке — кодирование категориального признака с m возможными значениями с помощью m бинарных признаков. Каждый бинарный признак соответствует одному из возможных значений категориального признака и является индикатором того, что на данном объекте он принимает данное значение. Данный подход иногда называют one-hot-кодированием. Воспользуйтесь им, чтобы перекодировать признаки LocationNormalized и ContractTime. Он уже реализован в классе sklearn.feature_extraction.DictVectorizer. Пример использования:

In [2]:
# from sklearn.feature_extraction import DictVectorizer
# enc = DictVectorizer()
# X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
# X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [3]:
import pandas as pd

In [50]:
data = pd.read_csv('salary-train.csv')

##### 2. Проведите предобработку:

In [51]:
#Приведите тексты к нижнему регистру (text.lower())
data['FullDescription'] = data['FullDescription'].str.lower()
data['LocationNormalized'] = data['LocationNormalized'].str.lower()

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова. Для такой замены в строке text подходит следующий вызов: re.sub('[^a-zA-Z0-9]', ' ', text). Также можно воспользоваться методом replace у DataFrame, чтобы сразу преобразовать все тексты:

In [52]:
data['FullDescription'] = data['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [53]:
vectorizer = TfidfVectorizer(min_df=5)
X_vec = vectorizer.fit_transform(data['FullDescription'])

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'.

In [54]:
data['LocationNormalized'].fillna('nan', inplace=True)
data['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [55]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [44]:
from scipy.sparse import hstack

In [56]:
X_train = hstack([X_vec,X_train_categ])

In [57]:
y_train = data['SalaryNormalized']

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [58]:
 from sklearn.linear_model import Ridge

In [60]:
rid = Ridge(alpha=1, random_state=241)

In [61]:
rid.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

Постройте прогнозы для двух примеров из файла salary-test-mini.csv.

In [62]:
data_test = pd.read_csv('salary-test-mini.csv')

data_test['FullDescription'] = data_test['FullDescription'].str.lower()
data_test['LocationNormalized'] = data_test['LocationNormalized'].str.lower()
data_test['FullDescription'] = data_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True)

X_vec_test = vectorizer.transform(data_test['FullDescription'])
data_test['LocationNormalized'].fillna('nan', inplace=True)
data_test['ContractTime'].fillna('nan', inplace=True)
X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_test = hstack([X_vec_test,X_test_categ])
rid.predict(X_test)

array([56563.50327943, 37141.00207403])